In [4]:
%load_ext autoreload 
%autoreload 2
import os
from os.path import expanduser
home = expanduser("~")
os.chdir(os.path.join(home,'Documents','repos','agu_data','agu_data'))
tablepath = os.path.join(home,'Documents/repos/agu_data/Notebook/Tables')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# AGU 2015: Exploratory analysis

Agu_Data
-------

I recently decided to scrap the AGU website [AGU](https://meetings.agu.org/) to look for
interesting pattern in the abstract. Here is a first analysis.


## Data 

In [5]:
from Data_Utils import *
import pandas as pd

In [6]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def presentation(time):
    time_beg = time.split('-')[0].split(':')[0]
    time_end = time.split('-')[1].split(':')[0]
    if int(time_end)-int(time_beg) == 0:
        return 'Oral'
    else:
        return 'Poster'
    
def clean_session(session):
    return ' '.join([f for f in session.split(' ') if f not in ['I','II','III','IV','V','Posters','(Half Session)']])
    
def extract_info(obj):
    ''' obj is a Paper object '''
    try:
        year = str(obj.date.split(',')[1].split(' ')[-1])
        day = obj.date.split(',')[0].strip()
        date = obj.date.split(',')[1].strip()
        nb_authors = len(obj.authors)
        place = obj.place.split('-')[0].strip()
        room = obj.place.split('-')[-1].strip()
        ref = len(obj.reference)
        section = obj.section.strip().encode('utf-8')
        session = clean_session(obj.session.strip()).encode('utf-8')
        pres = presentation(obj.time)
        title_words = tokenizer.tokenize(repr(obj.title[0]).lower())
        invited = int('invited' in title_words)
        lentitle = len(title_words)-1 # to remove the 'u' from unicode
        return [day,date,nb_authors,place,room,ref,section,session,pres,invited,lentitle,year]
    except:
        return ['']
    
columns = ['day','date','nb_authors','place','room','ref','section','session','pres','invited','lentitle','year']
df = pd.DataFrame(columns = columns)
for year in [2014,2015]:
    data = get_all_data('agu'+str(year))
    df = df.append(pd.DataFrame(map(extract_info,data),columns = columns).dropna())
    df.to_csv(os.path.join(tablepath,'agu_exploratory_tmp.csv'))
    

# Rapid inspection

In [13]:
len(df[df.year == '2015']),len(df[df.year == '2014'])

(22204, 23291)

#  Ok

In [334]:
js = "<script type='text/javascript' src='https://public.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 654px; height: 742px;'><noscript><a href='#'><img alt='ExploratoryAnalysisSection ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ex&#47;ExploratoryAnalysisAGUSection&#47;ExploratoryAnalysisSection&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz' width='654' height='742' style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='ExploratoryAnalysisAGUSection&#47;ExploratoryAnalysisSection' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ex&#47;ExploratoryAnalysisAGUSection&#47;ExploratoryAnalysisSection&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='showVizHome' value='no' /><param name='showTabs' value='y' /><param name='bootstrapWhenNotified' value='true' /></object></div>"
from IPython.display import HTML
HTML(js)

In [3]:
df0 = df[['session','section','day','year']]

NameError: name 'df' is not defined